<a href="https://colab.research.google.com/github/diamantron/seq2sql/blob/master/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Description: Predict stock price using a neural network
# =======================================================

In [13]:
# Import libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [16]:
# Get the stock quote
df = web.DataReader('AMZN', data_source='yahoo', start='2000-01-01', end='2020-12-01')

# Show the data
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,89.562500,79.046875,81.500000,89.375000,16117600,89.375000
2000-01-04,91.500000,81.750000,85.375000,81.937500,17487400,81.937500
2000-01-05,75.125000,68.000000,70.500000,69.750000,38457400,69.750000
2000-01-06,72.687500,64.000000,71.312500,65.562500,18752000,65.562500
2000-01-07,70.500000,66.187500,67.000000,69.562500,10505400,69.562500
...,...,...,...,...,...,...
2020-11-24,3134.250000,3086.260010,3100.500000,3118.060059,3602100,3118.060059
2020-11-25,3198.000000,3140.260010,3141.870117,3185.070068,3790400,3185.070068
2020-11-27,3216.189941,3190.050049,3211.260010,3195.340088,2392900,3195.340088
